In [1]:
%pip install numpy openai pydub whisper transformers
%pip install python-dotenv


  Using cached numpy-2.1.3-cp311-cp311-macosx_14_0_x86_64.whl (6.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 3.4 MB/s eta 0:00:0000:010:01
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 2.0 MB/s eta 0:00:0000:0100:01
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_10_9_x86_64.whl (184 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.7/287.7 kB 2.6 MB/s eta 0:00:0000:0100:01
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.0 MB/s eta 0:00:0000:0100:01
  Using cached safetensors-0.4.5-cp311-cp311-macosx_10_12_x86_64.whl (392 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Using cached charset_normalizer-3.4.0-cp311-cp311-macosx_10_9_x86_64.whl (124 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
N

In [3]:
# Real-Time Red Sales Agent
# A notebook to demonstrate real-time transcription with Whisper and actionable insights using GPT.

# Import required libraries
import os
import time
import numpy as np
import openai
from pydub import AudioSegment
from pydub.playback import play
from openai import OpenAI
import tempfile
import threading
import queue
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
# Set up API keys and environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define constants
TEMP_DIR = tempfile.mkdtemp()
CHUNK_DURATION = 3  # Duration of audio chunks in seconds
WHISPER_MODEL_SIZE = "small"  # Options: tiny, small, medium, large
GPT_MODEL = "gpt-4"

# Initialize the Whisper model
client = OpenAI()

# Queue for audio chunks
audio_queue = queue.Queue()

In [5]:
# Define audio streaming and recording logic
def record_stream_to_file(stream, output_dir=TEMP_DIR):
    """
    Record stream audio to files as .mp3 in CHUNK_DURATION second chunks.
    Args:
        stream: Audio input stream (mocked as example)
        output_dir (str): Directory to save audio chunks
    """
    print("[INFO] Starting audio recording...")
    for i, chunk in enumerate(stream):
        file_path = os.path.join(output_dir, f"chunk_{i}.mp3")
        chunk.export(file_path, format="mp3")
        print(f"[INFO] Saved chunk: {file_path}")
        audio_queue.put(file_path)
        time.sleep(CHUNK_DURATION)  # Simulate real-time streaming delay


In [6]:
# Define transcription logic
def transcribe_file(model_str, file_path):
    """
    Transcribe an audio file using Whisper.
    Args:
        model: Whisper model instance
        file_path (str): Path to the audio file
    Returns:
        str: Transcription text
    """
    try:
        transcription = client.audio.transcriptions.create(
            model=model_str, 
            file=file_path
        )
        print(f"[TRANSCRIPTION] {transcription.text}")
        return transcription.text
    except Exception as e:
        print(f"[ERROR] Transcription failed for {file_path}: {e}")
        return ""


In [7]:
# Define GPT streaming logic
def gpt_streaming_logic(transcription):
    """
    Stream transcription to GPT and print insights in real-time.
    Args:
        transcription (str): Input transcription text
    """
    try:
        response = openai.Completion.create(
            engine=GPT_MODEL,
            prompt=f"Provide actionable sales insights for the following transcript:\n{transcription}",
            max_tokens=150,
            stream=True,
        )
        print("[GPT INSIGHTS]")
        for chunk in response:
            if "choices" in chunk and "text" in chunk["choices"][0]:
                print(chunk["choices"][0]["text"], end="")
    except Exception as e:
        print(f"[ERROR] GPT streaming failed: {e}")

In [ ]:
# Mocking an audio stream (replace with actual stream in production)
def generate_mock_stream():
    """
    Generate a mock audio stream using pydub.
    Returns:
        list: Simulated audio chunks as AudioSegment objects
    """
    audio = AudioSegment.from_file("example_audio.mp3", format="mp3")
    chunks = [audio[i * 1000 * CHUNK_DURATION:(i + 1) * 1000 * CHUNK_DURATION] for i in range(len(audio) // (1000 * CHUNK_DURATION))]
    return chunks




In [ ]:
# Threaded transcription and GPT logic
def transcription_and_gpt_pipeline():
    """
    Process audio chunks from the queue for transcription and GPT response.
    """
    while True:
        if not audio_queue.empty():
            file_path = audio_queue.get()
            transcription = transcribe_file('whisper-1', file_path)
            if transcription:
                gpt_streaming_logic(transcription)

In [ ]:
# Main Execution
if __name__ == "__main__":
    # Start transcription and GPT pipeline in a separate thread
    threading.Thread(target=transcription_and_gpt_pipeline, daemon=True).start()
    
    # Simulate real-time audio recording and streaming
    audio_stream = generate_mock_stream()  # Replace with actual stream in production
    record_stream_to_file(audio_stream)
